In [1]:
import requests
import json
from bs4 import BeautifulSoup
from kinopoisk_unofficial.kinopoisk_api_client import KinopoiskApiClient
from kinopoisk_unofficial.request.films.film_request import FilmRequest
from kinopoisk_unofficial.request.films.box_office_request import BoxOfficeRequest
import time
from selenium import webdriver
import pandas as pd

In [2]:
browser = webdriver.Chrome('/Users/kosarenok/Desktop/chromedriver_mac64/chromedriver')

url_api = "https://kinopoiskapiunofficial.tech/api/v2.2/films/"
url = "https://www.kinopoisk.ru/awards/bafta/"

browser.get(url)

# r = requests.get(url)

soup = BeautifulSoup(browser.page_source, 'lxml')

/var/folders/n0/z4q56_291yl18hx7bzl2b5b40000gn/T/ipykernel_34394/4049708597.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('/Users/kosarenok/Desktop/chromedriver_mac64/chromedriver')


In [3]:
links = []
films_raw_data = []
for a in soup.findAll('a', class_='all'):
    links.append(a['href'].split('/')[2])

In [4]:
links.pop(0)

'www.bafta.org'

In [5]:
api_client = KinopoiskApiClient('9eaa3824-1c1b-41ee-850e-e6b20e2ed3fc')

counter = 0

for i in range(0,len(links)):
    counter += 1
    if counter <= 10:
        request = FilmRequest(int(links[i]))
        response = api_client.films.send_film_request(request)
        films_raw_data.append(response.film)
    else:
        time.sleep(2)
        counter = 0

In [39]:
def find_budget(response):
    for x in range(len(response.items)):
        if response.items[x].type == 'WORLD':
            return response.items[x]
        elif [x for x in range(len(response.items)) if response.items[x].type == 'WORLD'] == []:
            if response.items[x].type == 'BUDGET':
                return response.items[x]
            elif [x for x in range(len(response.items)) if response.items[x].type == 'BUDGET'] == []:
                return None

In [45]:
films_budget_data = []

counter = 0

for j in range(0,len(films_raw_data)):
    counter += 1
    if counter <= 10:
        request = BoxOfficeRequest(films_raw_data[j].kinopoisk_id)
        response = api_client.films.send_box_office_request(request)
        try:
            film_budget = {
                'kino_id' : films_raw_data[j].kinopoisk_id,
                'type' : find_budget(response).type,
                'amount' : find_budget(response).amount,
                'Currency' : find_budget(response).currency_code
            }
        except AttributeError as e:
            film_budget = {
                'kino_id' : films_raw_data[j].kinopoisk_id,
                'type' : None,
                'amount' : None,
                'Currency' : None
            }

        films_budget_data.append(film_budget)
    else:
        time.sleep(2)
        counter = 0

In [49]:
films_award_data = []

headers = {'X-API-KEY': '9eaa3824-1c1b-41ee-850e-e6b20e2ed3fc'}
counter = 0

for k in range(0,len(films_raw_data)):
    counter += 1
    if counter <= 10:
        request = requests.get(f'https://kinopoiskapiunofficial.tech/api/v2.2/films/{films_raw_data[k].kinopoisk_id}/awards', headers=headers)
        response = request.json()
        film_awards = {
                'kino_id' : films_raw_data[k].kinopoisk_id,
                'name' : 'Британская академия'
            }
        for t in [x for x in range(len(response['items'])) if response['items'][x]['name'] == 'Британская академия']:
            film_awards[response['items'][t]['nominationName']] = response['items'][t]['win']
            film_awards['year'] = response['items'][t]['year']
            films_award_data.append(film_awards)
    else:
        time.sleep(2)
        counter = 0

In [50]:
films_award_dataframe = pd.DataFrame(films_award_data)
films_award_dataframe = films_award_dataframe.drop_duplicates()
films_dataframe = pd.DataFrame(films_raw_data)
films_budget_dataframe = pd.DataFrame(films_budget_data)

In [51]:
films_budget_dataframe.rename(columns = {'kino_id':'kinopoisk_id'}, inplace=True)
films_award_dataframe.rename(columns = {'kino_id':'kinopoisk_id'}, inplace=True)

In [52]:
films_with_budget = films_dataframe.merge(films_budget_dataframe, how='outer', on='kinopoisk_id')

In [53]:
all_films_data = films_with_budget.merge(films_award_dataframe, how='outer', on='kinopoisk_id')

In [54]:
all_films_data.to_excel("Kinopoisk_bafta_parsed_data.xlsx", sheet_name='raw_data') 